In [19]:
# import requests
# from urllib import parse
# import pprint
# pp = pprint.PrettyPrinter(indent=4)
# import pandas as pd
# import json
# import numpy as np
# import sys
# from copy import deepcopy

In [113]:
import import_ipynb
from Game_to_Dataframe import make_df
from Using_API import load_data
import json
import numpy as np
import pandas as pd
from collections import Counter
import time
import requests
import csv

In [21]:
pd.set_option('display.max_rows', None)

# 챔피언, 특성(시너지), 아이템 json 파일 불러오기

with open("ko_kr.json", 'r') as file:
    data = json.load(file)

In [27]:
with open("item data.json", 'r') as file:
    data = json.load(file)

## 챔피언 이름 영어 -> 한국어

In [28]:
def name_eng_kr(data_champions) :
    champ_dict = dict()
    for i in range(data_champions.shape[0]) :
        champ_name = data_champions.apiName[i]
        champ_name_kr = data_champions.name[i]
        champ_dict[champ_name] = champ_name_kr
    return champ_dict

In [29]:
list_champions = pd.DataFrame(data['setData']).loc[0,:]['champions'].copy()
data_champions = pd.DataFrame(list_champions)
champ_kr = name_eng_kr(data_champions)

{'TFT7_Vladimir': '블라디미르',
 'TFT7_Nidalee': '니달리',
 'TFT7_Senna': '세나',
 'TFT7_Sett': '세트',
 'TFT7_Gnar': '나르',
 'TFT7_Sejuani': '세주아니',
 'TFT7_Ezreal': '이즈리얼',
 'TFT7_Skarner': '스카너',
 'TFT7_Yone': '요네',
 'TFT7_Varus': '바루스',
 'TFT7_LeeSin': '리 신',
 'TFT7_Shyvana': '쉬바나',
 'TFT7_Hecarim': '헤카림',
 'TFT7_Soraka': '소라카',
 'TFT7_Karma': '카르마',
 'TFT7_Zoe': '조이',
 'TFT7_Yasuo': '야스오',
 'TFT7_Nunu': '누누와 윌럼프',
 'TFT7_Bard': '바드',
 'TFT7_IvernMinion': '조이의 데이지',
 'TFT7_Xayah': '자야',
 'TFT7_Qiyana': '키아나',
 'TFT7_Sylas': '사일러스',
 'TFT7_Leona': '레오나',
 'TFT7_Twitch': '트위치',
 'TFT6_TheGoldenEgg': '황금 알',
 'TFT7_AoShin': '아오 신',
 'TFT7_DragonPurple': '사이펜',
 'TFT7_DragonGreen': '시오유',
 'TFT7_DragonGold': '이다스',
 'TFT7_DragonBlue': '다에야',
 'TFT7_JadeStatue': '비취 조각상',
 'TFT7_AurelionSol': '아우렐리온 솔',
 'TFT7_AquaticDragon': '솜',
 'TFT7_Taliyah': '탈리야',
 'TFT7_Malphite': '말파이트',
 'TFT7_Zac': '자크',
 'TFT7_Aphelios': '아펠리오스',
 'TFT7_Rell': '렐',
 'TFT7_Zyra': '자이라',
 'TFT7_Kaisa': '카이사',
 'TFT7_Seraphi

# traits(특성) json 파일 불러오기

In [514]:
with open("item data.json", 'r') as file:
    data = json.load(file)

In [515]:
data_traits = pd.DataFrame(data['setData']).loc[0,:]['traits']
traits = pd.DataFrame(data_traits)
traits

,apiName,desc,effects,icon,name
0,Set7_Cannoneer,5번째 공격마다 대포를 발사하고 대상 주변에 폭발을 일으켜 물리 피해를 입힙니다.<...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_5_Cannoneer.tex,포병대
1,Set7_Tempest,@DelayTime@초 후에 전장에 벼락이 내립니다. 적이 @StunDuration...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_7_Tempest.tex,폭풍
2,Set7_Assassin,고유: 전투 시작 시 적의 후방으로 도약합니다.<br><br>암살자는 추가 치명타 ...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_6_Assassin.tex,암살자
3,Set7_Prodigy,(@MinUnits@) 뇸뇸이는 게임마다 무작위 특성을 얻습니다. 이전 조련사를 소...,"[{'maxUnits': 25000, 'minUnits': 3, 'style': 4...",ASSETS/UX/TraitIcons/Trait_Icon_Prodigy.TFT_Se...,신동
4,Set7_Dragon,고유: 2개의 팀 슬롯이 필요합니다. <br><br>용은 특성 중첩을 3 증가시키고...,"[{'maxUnits': 1, 'minUnits': 1, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_7_Dragon.tex,용
5,Set7_Ragewing,고유: 마나를 분노로 전환합니다. 기본 공격 시 분노를 15 생성합니다. 스킬을 사...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_7_Ragewing.tex,분노날개
6,Set7_Guild,아군에게 고유 보너스를 부여합니다. 길드원은 두 배로 얻습니다. 전장의 길드 아군마...,"[{'maxUnits': 1, 'minUnits': 1, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_7_Guild.tex,길드
7,Set7_Cavalier,고유: 이동 시 대상에게 빠르게 돌격합니다.<br><br>기병대가 방어력과 마법 저...,"[{'maxUnits': 2, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_5_Cavalry.tex,기병대
8,Set7_Mage,요술사는 스킬을 두 번 사용하고 총 주문력이 조정됩니다.<br><br><expand...,"[{'maxUnits': 4, 'minUnits': 3, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_2_Mage.tex,요술사
9,Set7_Evoker,아군 또는 적이 스킬을 사용할 때마다 일깨우는 자가 마나를 얻습니다.<br><br>...,"[{'maxUnits': 2, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_7_Evoker.tex,일깨우는 자


# item 조합표 만들기

In [516]:
items = pd.DataFrame(data['items'])

In [529]:
items_data = pd.read_csv("item_list.csv", encoding = 'CP949')

In [530]:
items_season

,name,count,ave_score,win_rate,save_rate,game_count
0,최후의 속삭임,0,0,0,0,0
1,가고일 돌갑옷,0,0,0,0,0
2,기계유망주 상징,0,0,0,0,0
3,마법공학 생명검,0,0,0,0,0
4,B.F. 대검,0,0,0,0,0
5,구인수의 격노검,0,0,0,0,0
6,히라나의 창,0,0,0,0,0
7,선의 상징,0,0,0,0,0
8,영혼의 형상,0,0,0,0,0
9,황소부대 상징,0,0,0,0,0


## item raw data 에서 item 만 추출

In [34]:
def make_item_list(items) :
    n = len(items)
    basic = set([i for i in range(1,10)])
    data = pd.DataFrame(columns = items.columns)
    for i in range(n) :
        if items['id'][i] in basic :
            data = pd.concat([data, items.iloc[i:(i+1), :]])
        else :
            item_from = set(items['from'][i])
            if len(item_from) != 0 and item_from.issubset(basic) :
                data = pd.concat([data, items.iloc[i:(i+1), :]])
            
    return data

items_comb = make_item_list(items).reset_index(drop = True)

In [35]:
items_comb

,apiName,desc,effects,from,icon,id,name,unique
0,TFT_Item_RabadonsDeathcap,추가 주문력 %i:scaleAP% @BonusAP@,"{'AP': 20.0, '{d49caf5d}': 50.0}","[3, 3]",ASSETS/Maps/Particles/TFT/Item_Icons/Standard/...,33,라바돈의 죽음모자,False
1,TFT_Item_InfinityEdge,추가 공격력 %i:scaleAD% @BonusAD*100@% 및 추가 치명타 확률 ...,"{'AD': 0.10000000149011612, 'BonusAD': 0.15000...","[1, 9]",ASSETS/Maps/Particles/TFT/Item_Icons/Standard/...,19,무한의 대검,False
2,TFT7_Item_CavalierEmblemItem,장착 시 기병대 특성 획득<br><br><tftitemrules>[고유 - 중복 적...,{'Armor': 20.0},"[8, 5]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,2307,기병대 상징,True
3,TFT_Item_FrozenHeart,시작 마나 %i:scaleMana% @BonusMana@ 증가 <br><br>전투당...,"{'Armor': 20.0, 'HealthThreshold': 40.0, 'Mana...","[4, 5]",ASSETS/Maps/Particles/TFT/Item_Icons/Standard/...,45,수호자의 맹세,False
4,TFT8_Item_OxForceEmblemItem,장착 시 황소부대 특성 획득<br><br><tftitemrules>[고유 - 중복 ...,{'Armor': 20.0},"[8, 5]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,8004,황소부대 상징,True
5,TFT8_Item_ADMINEmblemItem,장착 시 자동방어체계 특성 획득<br><br><tftitemrules>[고유 - 중...,{'MagicResist': 20.0},"[6, 8]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,8007,자동방어체계 문장,True
6,TFT8_Item_AnimaSquadEmblemItem,장착 시 동물특공대 특성 획득<br><br><tftitemrules>[고유 - 중복...,{'AP': 10.0},"[8, 3]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,8000,동물특공대 상징,True
7,TFT8_Item_HeartEmblemItem,장착 시 선의 특성 획득<br><br><tftitemrules>[고유 - 중복 적용...,{'Mana': 15.0},"[8, 4]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,8016,선의 상징,True
8,TFT8_Item_InterPolarisEmblemItem,장착 시 레이저단 특성 획득<br><br><tftitemrules>[고유 - 중복 ...,{'AD': 0.10000000149011612},"[8, 1]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,8003,레이저단 상징,True
9,TFT8_Item_MascotEmblemItem,장착 시 마스코트 특성 획득<br><br><tftitemrules>[고유 - 중복 ...,{'Health': 150.0},"[8, 7]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,8008,마스코트 상징,True


## 시즌 정보 추출

In [36]:
# 시즌 정보
items_comb['season'] = 0 * len(items_comb)
for i in range(len(items_comb)) :
    season = items_comb['apiName'][i].split('_')[0]
    if season == 'TFT' :
        season = 1
    else :
        season = season[3:]
    items_comb.loc[i, ['season']] = int(season)
items_comb = items_comb.sort_values(by = ['season'], ascending = False).reset_index(drop = True)

## 조합표 만들기

In [423]:
items_mat

,0,1,2,3,4,5,6,7,8,9
0,0,B.F. 대검,곡궁,쓸데없이 큰 지팡이,여신의 눈물,쇠사슬 조끼,음전자 망토,거인의 허리띠,뒤집개,연습용 장갑
1,B.F. 대검,죽음의 검,거인 학살자,마법공학 총검,쇼진의 창,밤의 끝자락,피바라기,지크의 전령,레이저단 상징,무한의 대검
2,곡궁,거인 학살자,고속 연사포,구인수의 격노검,스태틱의 단검,거인의 결의,루난의 허리케인,즈롯 차원문,결투가 상징,최후의 속삭임
3,쓸데없이 큰 지팡이,마법공학 총검,구인수의 격노검,라바돈의 죽음모자,대천사의 지팡이,강철의 솔라리 펜던트,이온 충격기,모렐로노미콘,동물특공대 상징,보석 건틀릿
4,여신의 눈물,쇼진의 창,스태틱의 단검,대천사의 지팡이,푸른 파수꾼,수호자의 맹세,힘의 성배,구원,선의 상징,정의의 손길
5,쇠사슬 조끼,밤의 끝자락,거인의 결의,강철의 솔라리 펜던트,수호자의 맹세,덤불 조끼,가고일 돌갑옷,태양불꽃 망토,황소부대 상징,침묵의 장막
6,음전자 망토,피바라기,루난의 허리케인,이온 충격기,힘의 성배,가고일 돌갑옷,용의 발톱,서풍,자동방어체계 문장,수은
7,거인의 허리띠,지크의 전령,즈롯 차원문,모렐로노미콘,구원,태양불꽃 망토,서풍,워모그의 갑옷,마스코트 상징,방패파괴자
8,뒤집개,레이저단 상징,결투가 상징,동물특공대 상징,선의 상징,황소부대 상징,자동방어체계 문장,마스코트 상징,전략가의 왕관,무법자 상징
9,연습용 장갑,무한의 대검,최후의 속삭임,보석 건틀릿,정의의 손길,침묵의 장막,수은,방패파괴자,무법자 상징,도적의 장갑


In [545]:
items_mat = [[0 for _ in range(10)] for _ in range(10)]
item_dict = {"B.F. 대검" : 1, 
             "곡궁" : 2, 
             "쓸데없이 큰 지팡이" : 3, 
             "여신의 눈물" : 4, 
             "쇠사슬 조끼" : 5, 
             "음전자 망토" : 6, 
             "거인의 허리띠" : 7, 
             "뒤집개" : 8, 
             "연습용 장갑" : 9}
items_season = pd.DataFrame(columns = items_comb.columns)
for i in range(len(items_comb)) :
    name = items_comb['name'][i]
    code = items_comb['id'][i]
    c = items_comb['from'][i]
    if not items_comb['from'][i] : # 기본템일 때
        items_mat[code][0] = name
        items_mat[0][code] = name
        items_season = pd.concat([items_season, items_comb.iloc[i:(i+1), :]])
    else: # 완성템일 때
        if items_mat[c[0]][c[1]] == 0 or items_mat[c[1]][c[0]] == 0 :
            items_mat[c[0]][c[1]] = name
            items_mat[c[1]][c[0]] = name
            items_season = pd.concat([items_season, items_comb.iloc[i:(i+1), :]])
            items_dict[name] = code
            
items_mat = pd.DataFrame(items_mat)

## 플레이어 당 챔피언 펼치기

In [547]:
df = array[:5]

In [548]:
match_df = make_df(df[0], 'tft8', data)

In [549]:
match_df.change_augment_name()
match_df.make_new_game_result()

,Game_id,Player_id,Placement,Time,Last_round,Damage_player,Gold_left,augments,traits,num_units,tier_current,units,unit_tier,item_name
0,KR_6339726436,ncOjcEhSOF6CLRCNnzXj4gmCjsvMHlcOc3P_KyIeqKiCHv...,3,2011.834839,35,124,0,"[곱빼기, 금의환향, 싸움꾼 왕관]","[에이스, 방패대, 싸움꾼, 메카:프라임, 마스코트, 위협]","[1, 2, 2, 3, 2, 2]","[1, 1, 1, 1, 1, 1]","[잭스, 알리스타, 아우렐리온 솔, 세트, 누누와 윌럼프, 우르곳, 우르곳, 레오나...","[2, 2, 2, 2, 2, 1, 1, 1, 2]","[[None, None, None], [None, None, None], [힘의 성..."
1,KR_6339726436,EsYWRfvAO-tplAzGByZjHKiCwzZ5t53D3y5NqynjZml4V0...,8,1483.337402,26,44,0,"[사냥의 전율 II, 금의환향, 행운의 장갑]","[에이스, 동물특공대, 익살꾼, 위협]","[1, 5, 3, 1]","[1, 2, 2, 1]","[나서스, 징크스, 베인, 리븐, 조이, 미스 포츈, 에코, 아우렐리온 솔]","[2, 1, 1, 2, 2, 2, 1, 2]","[[None, None, None], [None, None, None], [None..."
2,KR_6339726436,07PlhgM9xBSq4HMmkucGXlY2lcsyrmGrmIDB3RDWYup5x-...,5,1707.766479,30,72,0,"[조준경 부착 I, 우주의 방어막, 찬란한 유물]","[정찰단, 위협]","[4, 3]","[3, 1]","[애쉬, 이즈리얼, 베인, 초가스, 람머스, 카이사, 자크]","[2, 2, 2, 2, 2, 2, 1]","[[None, None, None], [장난꾸러기의 장갑, 공정의 주먹, 태양빛 망..."
3,KR_6339726436,cFFXEYQAA8uMfBk38a6cJ7bE_890irlR7d-0gjvbhQWcRJ...,2,2145.446289,38,130,0,"[레이저단 문장, 한밤의 서곡, 뜻밖의 횡재++]","[에이스, 타락, 결투가, 해커, 익살꾼, 레이저단, 위협]","[1, 1, 2, 2, 2, 6, 2]","[1, 1, 1, 1, 1, 2, 1]","[야스오, 조이, 세나, 에코, 제드, 세주아니, 우르곳, 모데카이저, 피들스틱]","[2, 2, 2, 2, 2, 2, 2, 2, 2]","[[None, None, None], [None, None, None], [서풍, ..."
4,KR_6339726436,cUWlMbESaYC74JUu32OoaeCInvARxjIE1sRYcD59NEpupH...,1,2145.446289,38,172,1,"[곱빼기, 승리의 장막, 레벨 업!]","[에이스, 병기고, 타락, 특등사수, 메카:프라임, 황소부대, 위협]","[4, 1, 1, 2, 3, 2, 1]","[2, 1, 1, 1, 1, 1, 1]","[드레이븐, 미스 포츈, 비에고, 사미라, 세트, 아펠리오스, 레오나, 모데카이저,...","[1, 2, 1, 3, 2, 2, 2, 2, 2]","[[None, None, None], [쇼진의 창, None, None], [Non..."
5,KR_6339726436,pCawJHbgwNVANS_qynI9sPUvAGL2X-zzP4nxD6kwTpT-nL...,4,2017.380981,35,132,1,"[맑은 정신, 레이저단 진형, 찬란한 유물]","[병기고, 싸움꾼, 특등사수, 결투가, 해커, 익살꾼, 레이저단]","[1, 2, 2, 2, 2, 2, 6]","[1, 1, 1, 1, 1, 1, 2]","[애쉬, 레넥톤, 야스오, 조이, 세나, 에코, 제드, 세주아니, 아펠리오스]","[1, 1, 1, 1, 2, 2, 2, 2, 2]","[[None, None, None], [None, None, None], [쓸데없이..."
6,KR_6339726436,FfJmwRP5WPSn0TsMfQCCItUh6aswZacT1c4QitEfyAsNGr...,7,1551.922485,27,88,9,"[부익부, 회피, 저주받은 왕관]","[에이스, 병기고, 타락, 특등사수, 엄호대, 메카:프라임, 황소부대, 위협]","[1, 1, 1, 2, 2, 3, 2, 3]","[1, 1, 1, 1, 1, 1, 1, 1]","[오공, 잭스, 알리스타, 사미라, 자크, 세트, 우르곳, 아펠리오스, 피들스틱, ...","[2, 2, 2, 1, 2, 1, 2, 1, 1, 1]","[[None, None, None], [None, None, None], [None..."
7,KR_6339726436,qjLx67TBW6saH8kRUyqASYjfaoACqH6z_DISFKdJhBz41U...,6,1617.824585,28,78,1,"[휴대용 대장간, 우주의 방어막, 큰손]","[민간인, 기상캐스터, 위협]","[1, 1, 5]","[1, 1, 1]","[초가스, 람머스, 람머스, 벨베스, 아우렐리온 솔, 자크, 잔나]","[3, 2, 2, 2, 1, 2, 1]","[[None, None, None], [이온 충격기, None, None], [No..."


In [550]:
def unfold_df(match_json) :
    unfolded_df = pd.DataFrame(columns = ['Game_id', 'Player_id', 'Placement', 'augments', 'traits', 'num_units',
       'tier_current', 'character_id', 'champ_name', 'items', 'item_name'])
    for i in range(len(match_json)) :
        match_df = make_df(match_json[i], 'tft8', data)
        match_df.change_augment_name()
        a1 = pd.concat([match_df.make_unit_result(i) for i in range(8)])
        a2 = match_df.make_new_game_result().drop(['Time', 'Last_round', 'Damage_player', 'Gold_left','units', 'item_name', 'unit_tier'], axis = 1)
        unfolded_df = pd.concat([unfolded_df, pd.merge(a2, a1, on = ['Game_id', 'Player_id'])], ignore_index = True)
    return unfolded_df

In [551]:
unfolded_df = unfold_df(df)

## 아이템 -> 조합 아이템

In [587]:
def call_item(item_name) :
    season_list = list(items_season.name)
    if item_name in season_list :
        temp = items_season[items_season['name'] == item_name].iloc[0]
        if temp['from']:
            item_from = sorted(temp['from'])
            return list(items_season.loc[items_season['id'] == item_from[0], 'name'].values) + list(items_season.loc[items_season['id'] == item_from[1], 'name'].values)
        else :
            return [item_name , None]
    else :
        return [None, None]

## 아이템 시너지 데이터프레임 만들기

In [588]:
item_df = pd.read_csv("item_result.csv", encoding = 'CP949')

In [595]:
def add_item_column(item_df) :
    item_df[['item1', 'item2']] = None
    name_list = list(item_df.name)
    for i in range(len(item_df)) :
        item_list = call_item(name_list[i])
        item_df.loc[i, 'item1'] = item_list[0]
        item_df.loc[i, 'item2'] = item_list[1]

# 증강체

In [72]:
def make_augment_list(augments) :
    data = pd.DataFrame(columns = items.columns)
    for i in range(len(augments)) :
        if 'Augment' in augments.loc[i,'apiName'] :
            data = pd.concat([data, pd.DataFrame(items.iloc[i,:]).T], axis = 0)
            
    return data

def aug_eng_kr(augments) :
    aug_dict = dict()
    for i in augments.index :
        aug_dict[augments.loc[i,'apiName']] = augments.loc[i,'name']
    return aug_dict

augments = make_augment_list(items)
aug_dict = aug_eng_kr(augments)

# load API key from txt file

In [12]:
def getAPIkey():
    f = open("./riot_api.txt","r")
    return f.read()

In [13]:
api_key = getAPIkey()

In [14]:
api_key

'RGAPI-5f78ebd4-6f3e-4f83-94d6-aa9f59e4b01f'

In [18]:
# request headers
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

In [19]:
# response body
# {
#     "id": "GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U",
#     "accountId": "YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4",
#     "puuid": "i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA",
#     "name": "예비역협회장",
#     "profileIconId": 5031,
#     "revisionDate": 1667979235000,
#     "summonerLevel": 97
# }

# 소환사 정보 구현하기

## 소환사명, puuid, accountid, id 통해서 소환사 검색

### 소환사명으로 검색

In [20]:
def summoner_v1_by_summoner_name(summonerName):
    encodingSummonerName = parse.quote(summonerName)
    url = f"https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/{encodingSummonerName}"
    return requests.get(url, headers=request_header).json()

In [21]:
pp.pprint(summoner_v1_by_summoner_name("예비역협회장"))

{   'accountId': 'YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4',
    'id': 'GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U',
    'name': '예비역협회장',
    'profileIconId': 1158,
    'puuid': 'i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA',
    'revisionDate': 1675055864000,
    'summonerLevel': 98}


### Account id 로 검색

In [22]:
def summoner_v1_by_accountId(accountId):
    url = f"https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-account/{accountId}"
    return requests.get(url, headers=request_header).json()

In [23]:
pp.pprint(summoner_v1_by_accountId("YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4"))

{   'accountId': 'YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4',
    'id': 'GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U',
    'name': '예비역협회장',
    'profileIconId': 1158,
    'puuid': 'i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA',
    'revisionDate': 1675055864000,
    'summonerLevel': 98}


### puuid 로 검색

In [24]:
def summoner_v1_by_puuid(puuid):
    url = f"https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-puuid/{puuid}"
    return requests.get(url, headers=request_header).json()

In [25]:
pp.pprint(summoner_v1_by_puuid('i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA'))

{   'accountId': 'YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4',
    'id': 'GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U',
    'name': '예비역협회장',
    'profileIconId': 1158,
    'puuid': 'i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA',
    'revisionDate': 1675055864000,
    'summonerLevel': 98}


### id 로 검색

In [26]:
def summoner_v1_summoner_name(id):
    url = f"https://kr.api.riotgames.com/tft/summoner/v1/summoners/{id}"
    return requests.get(url, headers=request_header).json()

In [27]:
pp.pprint(summoner_v1_summoner_name("GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U"))

{   'accountId': 'YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4',
    'id': 'GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U',
    'name': '예비역협회장',
    'profileIconId': 1158,
    'puuid': 'i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA',
    'revisionDate': 1675055864000,
    'summonerLevel': 98}


# puuid 에서 Match 로드

In [31]:
def match_v1_get_list_match_id(puuid):
    url = f"https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?start=0&count=50"
    return requests.get(url, headers=request_header).json()

In [32]:
match_v1_get_list_match_id('i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA')

['KR_6320812686',
 'KR_6287331764',
 'KR_6287251310',
 'KR_6275904427',
 'KR_6275849871',
 'KR_6275802979',
 'KR_6260284565',
 'KR_6260233775',
 'KR_6260192217',
 'KR_6256620336',
 'KR_6256543857',
 'KR_6239015316',
 'KR_6238978807',
 'KR_6238670316',
 'KR_6238657553',
 'KR_6238247398',
 'KR_6238152466',
 'KR_6237634578',
 'KR_6237625221',
 'KR_6237595884',
 'KR_6234266868',
 'KR_6234230264',
 'KR_6234185597',
 'KR_6233760399',
 'KR_6233561813',
 'KR_6233518827',
 'KR_6233264225',
 'KR_6233235821',
 'KR_6233220407',
 'KR_6233203056',
 'KR_6233183228',
 'KR_6232211117',
 'KR_6232150713',
 'KR_6229489564',
 'KR_6229481590',
 'KR_6229473538',
 'KR_6229461216',
 'KR_6219216312',
 'KR_6216559852',
 'KR_6210966508',
 'KR_6208570199',
 'KR_6208370312',
 'KR_6203295104',
 'KR_6202775387',
 'KR_6198887107',
 'KR_6198247318',
 'KR_6198100416',
 'KR_6198056901',
 'KR_6196640803',
 'KR_6189441839']

# Match data 불러오기

In [898]:
def load_match(matchid) :
    url = f"https://asia.api.riotgames.com/tft/match/v1/matches/{matchid}"
    return requests.get(url, headers=request_header).json()

In [899]:
match = load_match('KR_6208570199')

In [900]:
match.keys()

dict_keys(['metadata', 'info'])

In [999]:
match_metadata = match['metadata']
match_info = match['info']

match_metadata.keys()   # dict_keys(['data_version', 'match_id', 'participants'])
match_info.keys()
# dict_keys(['game_datetime', 'game_length', 'game_version', 'participants', 'queue_id',
#            'tft_game_type', 'tft_set_core_name', 'tft_set_number'])
match_info['participants'][0].keys()
# dict_keys(['augments', 'companion', 'gold_left', 'last_round', 'level', 'placement',
#            'players_eliminated', 'puuid', 'time_eliminated', 'total_damage_to_players', 'traits', 'units'])

dict_keys(['augments', 'companion', 'gold_left', 'last_round', 'level', 'placement', 'players_eliminated', 'puuid', 'time_eliminated', 'total_damage_to_players', 'traits', 'units'])

In [1004]:
match_result = pd.DataFrame(match_info['participants'])

In [1005]:
pd.DataFrame(match_result['units'][1])

,character_id,itemNames,items,name,rarity,tier
0,TFT7_Yone,[],[],,1,2
1,TFT7_Yone,[],[],,1,2
2,TFT7_Zyra,[TFT_Item_StatikkShiv],[24],,1,2
3,TFT7_Jax,[],[],,1,2
4,TFT7_Sylas,"[TFT_Item_ZekesHerald, TFT_Item_PowerGauntlet]","[17, 79]",,2,1
5,TFT7_Olaf,[],[],,2,2
6,TFT7_DragonPurple,"[TFT_Item_TitansResolve, TFT_Item_GuardianAngel]","[25, 94]",,5,2
7,TFT7_Pantheon,[TFT_Item_TitansResolve],[25],,4,1


In [907]:
match_result['traits'][0]

[{'name': 'Set7_Astral',
  'num_units': 8,
  'style': 3,
  'tier_current': 3,
  'tier_total': 3},
 {'name': 'Set7_Bruiser',
  'num_units': 2,
  'style': 1,
  'tier_current': 1,
  'tier_total': 4},
 {'name': 'Set7_Dragon',
  'num_units': 1,
  'style': 1,
  'tier_current': 1,
  'tier_total': 6},
 {'name': 'Set7_Evoker',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 3},
 {'name': 'Set7_Mage',
  'num_units': 5,
  'style': 2,
  'tier_current': 2,
  'tier_total': 4},
 {'name': 'Set7_Shapeshifter',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 2},
 {'name': 'Set7_Shimmerscale',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 4},
 {'name': 'Set7_SpellThief',
  'num_units': 1,
  'style': 3,
  'tier_current': 1,
  'tier_total': 1},
 {'name': 'Set7_Swiftshot',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 4},
 {'name': 'Set7_Whispers',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 3}]

In [994]:
match_result.units[0]

[{'character_id': 'TFT7_Vladimir',
  'itemNames': ['TFT_Item_ThiefsGloves',
   'TFT_Item_Chalice',
   'TFT_Item_PowerGauntlet'],
  'items': [99, 46, 79],
  'name': 'Vladimir',
  'rarity': 0,
  'tier': 3},
 {'character_id': 'TFT7_Skarner',
  'itemNames': ['TFT_Item_IonicSpark'],
  'items': [36],
  'name': 'Skarner',
  'rarity': 0,
  'tier': 3},
 {'character_id': 'TFT7_Nidalee',
  'itemNames': [],
  'items': [],
  'name': 'Nidalee',
  'rarity': 0,
  'tier': 3},
 {'character_id': 'TFT7_Lux',
  'itemNames': ['TFT_Item_ArchangelsStaff',
   'TFT_Item_JeweledGauntlet',
   'TFT_Item_InfinityEdge'],
  'items': [34, 39, 19],
  'name': '',
  'rarity': 1,
  'tier': 3},
 {'character_id': 'TFT7_Sylas',
  'itemNames': ['TFT_Item_BrambleVest',
   'TFT_Item_GargoyleStoneplate',
   'TFT4_Item_OrnnEternalWinter'],
  'items': [55, 56, 9003],
  'name': '',
  'rarity': 2,
  'tier': 2},
 {'character_id': 'TFT7_Varus',
  'itemNames': ['TFT_Item_MadredsBloodrazor', 'TFT_Item_Quicksilver'],
  'items': [12, 69],

# Match load

In [2]:
data = list(pd.read_csv("game_dict.csv").game_id)

In [297]:
data[1500:]

['KR_6339726436',
 'KR_6339250581',
 'KR_6334105605',
 'KR_6340441276',
 'KR_6338778818',
 'KR_6333498400',
 'KR_6335862492',
 'KR_6321926692',
 'KR_6339733538',
 'KR_6338527623',
 'KR_6332418059',
 'KR_6335994932',
 'KR_6333874041',
 'KR_6337836384',
 'KR_6334547330',
 'KR_6327267079',
 'KR_6339992417',
 'KR_6338007502',
 'KR_6340965464',
 'KR_6331066770',
 'KR_6334663645',
 'KR_6341059048',
 'KR_6333898741',
 'KR_6326289245',
 'KR_6323893204',
 'KR_6340979614',
 'KR_6341257788',
 'KR_6324702758',
 'KR_6339334520',
 'KR_6335155825',
 'KR_6337879917',
 'KR_6340282796',
 'KR_6321910206',
 'KR_6337374222',
 'KR_6319913724',
 'KR_6335247799',
 'KR_6342399803',
 'KR_6336127025',
 'KR_6341211032',
 'KR_6333068925',
 'KR_6341536260',
 'KR_6335795783',
 'KR_6339372336',
 'KR_6325429782',
 'KR_6336148467',
 'KR_6332029749',
 'KR_6334410316',
 'KR_6336518820',
 'KR_6318844039',
 'KR_6338455073',
 'KR_6329472782',
 'KR_6333660928',
 'KR_6334556589',
 'KR_6333970986',
 'KR_6340856421',
 'KR_63393

In [6]:
import_data = load_data(api_key)

In [ ]:
array = []

for i in range(1500, len(data)):
    check = import_data.extract_game_result(data[i], world='asia')
    array.append(check)
    
    if not i%50:
        print(i)
        print(array[-1])
    time.sleep(2)
    

In [ ]:
import_data.extract_game_result(data[0], world='asia')

In [8]:
data[0]

'KR_6340805042'

In [16]:
len(array)

1778

In [17]:
# with open("1500.json", 'w') as outfile:
#     json.dump(array, outfile)